In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import time
from datetime import datetime
from datetime import timedelta
import pandas as pd
import json
import pymongo
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from IPython.display import clear_output

In [2]:
def create_ListofDict(url,view,totalshr,shrFb,shrTw,shrGp,txt):
    info = {
            "URL":url,
            "totalView": view,
            "totalShare": totalshr,
            "share_fb": shrFb,
            "share_tw": shrTw,
            "share_gp": shrGp,
            "text": txt
    }
    return info

In [3]:
def create_ListofDict_InsertMany(df):
    listofdict = []
    for c,lx in enumerate(df.URL,0):
        info = {
            "URL":lx,
            "totalView": int(df.totalView[c]),
            "totalShare": int(df.totalShare[c]),
            "share_fb": int(df.share_fb[c]),
            "share_tw": int(df.share_tw[c]),
            "share_gp": int(df.share_gp[c]),
            "text": df.text[c]
        }
        listofdict.append(info)
    return listofdict

In [4]:
def convertKtoNum(s):
    if  'K' in s:
        ss = s.split('K')
        n = float(ss[0])*1000
    else:
        n = int(s)
    return n

In [5]:
# Connect MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["BADS7101"]
q_c_urlList = mydb["URL_LIST_CRIME"]
q_p_urlList = mydb["URL_LIST_POLITIC_REMDUP"]
q_e_urlList = mydb["URL_LIST_ENTER"]
q_p2_urlList = mydb["POLITIC_NEWS_U"]

i_c_scrape = mydb["CRIME_NEWS"]
i_p_scrape = mydb["POLITIC_NEWS_U"]
i_e_scrape = mydb["ENTERTAIN_NEWS"]

### ข่าวอาชญากรรม

In [6]:
cursor1 = q_c_urlList.find()

In [7]:
df_crime = pd.DataFrame(cursor1)
df_crime.tail()

,_id,timestamp,URL,headline,newsdate
1875,5fa6caf852e52341a96beb2b,2020-11-07 23:27:27.954,https://www.dailynews.co.th/crime/777584,จับโจ๋ลวงด.ญ.14สติไม่ดีข่มขืน อ้างทักแชทมาหาเร...,2020-06-01 12:10:00
1876,5fa6caf852e52341a96beb2c,2020-11-07 23:27:27.954,https://www.dailynews.co.th/crime/777568,เด้ง8ตร.สภ.หนองจอกเซ่นคลิป เรียกเงินต่อรองก่อน...,2020-06-01 11:18:00
1877,5fa6caf852e52341a96beb2d,2020-11-07 23:27:27.954,https://www.dailynews.co.th/crime/777532,รักผ้าขาวม้ามาก'ครูหื่น'ยื่น5แสน ได้ประกัน-คลุ...,2020-05-31 22:59:00
1878,5fa6caf852e52341a96beb2e,2020-11-07 23:27:27.954,https://www.dailynews.co.th/crime/777515,หนุ่มขับรถขนเครื่องบินทุกข์ใจ นายสั่งแต่รับกรร...,2020-05-31 19:00:00
1879,5fa6caf852e52341a96beb2f,2020-11-07 23:27:27.954,https://www.dailynews.co.th/crime/777503,ช็อกแค่พลาดบอกรหัสOTP จากมีเงิน4แสน-เหลือ70บาท,2020-05-31 17:11:00


In [8]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
g = datetime.strptime(str(max(df_crime.newsdate)), '%Y-%m-%d %H:%M:%S')
mts = timedelta(days=90)
print('Min time:',min(df_crime.newsdate))
print('Max time',max(df_crime.newsdate))
print('Min filter time',g-mts)

ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)
Min time: 2020-05-31 17:11:00
Max time 2020-11-07 16:20:00
Min filter time 2020-08-09 16:20:00


In [9]:
df_crime = df_crime[df_crime.newsdate >= g-mts]
df_crime.head()

,_id,timestamp,URL,headline,newsdate
0,5fa6c15052e52341a96be3d8,2020-11-07 22:46:16.338,https://www.dailynews.co.th/crime/805599,ตร.จัดเต็มรับมือม็อบ! ทุ่ม316ล้านซื้ออุปกรณ์คุ...,2020-11-07 16:20:00
1,5fa6c15052e52341a96be3d9,2020-11-07 22:46:16.338,https://www.dailynews.co.th/crime/805562,220ศาลทั่วประเทศพร้อมแล้ว จัดไกล่เกลี่ยก่อนฟ้อง,2020-11-07 12:59:00
2,5fa6c15052e52341a96be3da,2020-11-07 22:46:16.338,https://www.dailynews.co.th/crime/805560,แจงไม่ห้ามตร.เล่นโซเชียล แต่ใช้ให้ถูกและระมัดร...,2020-11-07 12:49:00
3,5fa6c15052e52341a96be3db,2020-11-07 22:46:16.338,https://www.dailynews.co.th/crime/805547,ไว้ใจลงทุนให้เลี้ยงเป็ดนับพัน แสบจับขายเกลี้ยง...,2020-11-07 12:05:00
4,5fa6c15052e52341a96be3dc,2020-11-07 22:46:16.338,https://www.dailynews.co.th/crime/805545,รวบหนุ่มเมืองคอนหนีคดี12ปี หลอก2ด.ญ.พี่น้องขยี...,2020-11-07 11:56:00


df_crime.URL[0:100]  0-99<br>
df_crime.URL[100:300] 100-299

In [ ]:
i_c_scrape.delete_many({})

#df_crime.URL[0:2]

for c,each in enumerate(df_crime.URL,0):
    print('URL',c+1)
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'reader')))
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        process_data = create_ListofDict(each,view_,int(total_share),int(share_fb),int(share_tw),int(share_gp),text_)
        i_c_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out')
    driver.quit()
    clear_output(wait=True)

### ข่าวการเมือง

In [80]:
cursor2 = q_p_urlList.find()
df_politic_url = pd.DataFrame(cursor2)

In [81]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
#g = datetime.strptime(str(min(df_politic_url.newsdate)), '%Y-%m-%d %H:%M:%S')
#mts = timedelta(hours=6.8)
#print('Min filter',g+mts)
print('Min time:',min(df_politic_url.newsdate))
print('Max time',max(df_politic_url.newsdate))

ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)
Min time: 2020-07-01 09:54:00
Max time 2020-11-07 22:15:00


In [82]:
df_politic_url = df_politic_url[(df_politic_url.newsdate >= '2020-07-01 00:00:00')&(df_politic_url.newsdate <= '2020-08-10 00:00:00')]
df_politic_url.drop_duplicates(subset=['URL'], inplace=True, ignore_index=True)

cursorp2 = q_p2_urlList.find()
df_politic_exDetail = pd.DataFrame(cursorp2)
df_politic_exDetail.drop_duplicates(subset=['URL'], inplace=True, ignore_index=True)

df_politic_j = pd.merge(df_politic_url, df_politic_exDetail, how="left",on=['URL'])
df_politic_j = df_politic_j[df_politic_j._id_y.isnull()]

In [83]:
df_politic_j.tail(2)

,_id_x,timestamp,URL,headline,newsdate,_id_y,totalView,totalShare,share_fb,share_tw,share_gp,text
1518,5fb01e5f157004571de14365,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788738,เตือน!คนรุ่นใหม่ลุกฮือสู้เผด็จการ ล่า5หมื่นชื่...,2020-08-07 15:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1519,5fb01e5f157004571de14366,2020-11-09 18:05:03.083,https://www.dailynews.co.th/politics/788726,"""ชวน""เตือนสังคมไทยอย่าเกรงใจจนทำสิ่งไม่ถูกต้อง",2020-08-07 15:16:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
print('Min time:',min(df_politic_j.newsdate))
print('Max time',max(df_politic_j.newsdate))

Min time: 2020-08-07 15:16:00
Max time 2020-08-08 17:06:00


In [85]:
df_politic_j.shape

(32, 12)

In [86]:
#i_p_scrape.delete_many({})

reqTo = []

for c,each in enumerate(df_politic_j.URL,0):
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver,7).until(EC.presence_of_element_located((By.CLASS_NAME, 'reader')))
        myElem = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.ID, 'span_reader_ajax')))
        time.sleep(5)
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        print('URL',c+1,'Element',each,view_,convertKtoNum(total_share),share_fb,share_tw,share_gp,text_)
        process_data = create_ListofDict(each,view_,convertKtoNum(total_share),
                                         convertKtoNum(share_fb),convertKtoNum(share_tw),convertKtoNum(share_gp),text_)
        i_p_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out at row',c)
        reqTo.append(c)
    driver.quit()
    clear_output(wait=True)

URL 32 Element https://www.dailynews.co.th/politics/788726 1382 15 8 3 4 เมื่อวันที่ 7 ส.ค.  ที่สำนักงานคณะกรรมการป้องกันและปราบปรามการทุจริตแห่งชาติ(ป.ป.ช.) ในงานสัมมนาสาธารณะ หลักสูตรนักบริหารยุทธศาสตร์การป้องกันและปรับปรามการทุจริตรระดับสูง(นยปส.รุ่น11) นายชวน หลีกภัย ประธานรัฐสภา ปาฐกถาพิเศษเรื่อง “คุณธรรมโปร่งใส ยุค 4.0” ตอนหนึ่งว่า ธรรมาภิบาลภาคปฏิบัติ 6 ข้อ ซึ่งเกิดขึ้นหลังจากกรณีวิกฤติต้มยำกุ้ง ด้วยวิกฤติเหล่านั้นเอง ทำให้ต้องมีการศึกษาเรื่องธรรมาภิบาลขึ้นในไทยอย่างจริงจัง โดย 1 ปีหลังตนเป็นนายกรัฐมนตรี ได้มอบหมายให้ นายอภิสิทธิ์ เวชชาชีวะ และทีมงานรับผิดชอบศึกษาเรื่องนี้ เคาะออกมาเป็นนโยบาย โดยใช้คำว่า การบริหารจัดการที่ดีของภาครัฐและเอกชน ต่อมาคือคำว่าธรรมาภิบาล เพื่อป้องวิกฤติต่าง ๆ ไม่ให้เกิดขึ้นอีก โดยมีประเด็นสำคัญเช่น หลักนิติธรรมสำคัญที่สุด

นอกจากนั้นยังมีหลักกฎหมาย การมีส่วนร่วมประชาชน หลักความรับผิดชอบต่อสังคม แต่ตนเห็นว่าสำหรับสังคมไทยต้องเพิ่มเรื่องความไม่เกรงใจต่อสิ่งที่ไม่ถูกต้องอีกข้อหนึ่ง เพราะมีข้าราชการดีๆ  หรือนักการเมืองดีๆ หลายคนต้องติดคุก เพราะมาจากความเก

In [ ]:
reqTo

In [ ]:
# Remove duplicate and reinsert POLITIC_NEWS_U แล้วลบ POLITIC_NEWS เดิมออก เพื่อประหยัดเนื้อที่
#i_pp_scrape = mydb["POLITIC_NEWS_U"]
#cc = i_p_scrape.find()
#df_p_u = pd.DataFrame(cc)
#df_p_u = df_p_u.drop_duplicates(subset=['URL'],ignore_index=True)
#df_p_u.shape

In [ ]:
#df_p_u.tail()

In [ ]:
#i_pp_scrape.delete_many({})
#process_data = create_ListofDict_InsertMany(df_p_u)
#i_pp_scrape.insert_many(process_data)

### ข่าวบันเทิง

In [ ]:
cursor3 = q_e_urlList.find()

In [ ]:
df_entertain = pd.DataFrame(cursor3)
df_entertain.tail(3)

In [ ]:
print('ตรวจสอบวันแรกและสุดท้าย ก่อนตัด scope 3 เดือน (90 วัน)')
g = datetime.strptime(str(max(df_entertain.newsdate)), '%Y-%m-%d %H:%M:%S')
mts = timedelta(days=90)
print('Min time:',min(df_entertain.newsdate))
print('Max time',max(df_entertain.newsdate))
print('Min filter time',g-mts)

In [ ]:
df_entertain = df_entertain[df_entertain.newsdate >= g-mts]
df_entertain.tail()

In [ ]:
df_entertain.shape

In [ ]:
i_e_scrape.delete_many({})

reqTo = []

for c,each in enumerate(df_entertain.URL,0):
    driver = webdriver.Chrome(executable_path='selenium/chromedriver.exe')
    driver.get(each)
    try:
        myElem = WebDriverWait(driver,4).until(EC.visibility_of_element_located((By.CLASS_NAME, 'reader')))
        content=driver.page_source
        soup=BeautifulSoup(content,'lxml')
        div = soup.find("div", class_=lambda value: value and value=="share-box-mobile")
        h2 = div.find("h2", class_=lambda value: value and value=="heading")
        share_a_fb = div.find("a", class_=lambda value: value and value=="share-btn fb", href=True)
        share_a_tw = div.find("a", class_=lambda value: value and value=="share-btn tw", href=True)
        share_a_gp = div.find("a", class_=lambda value: value and value=="share-btn gp", href=True)
        div2 = soup.find("div", class_=lambda value: value and value=="article-subbox")
        view_div = div2.find("div", class_=lambda value: value and value=="upper")
        div3 = soup.find("div", class_=lambda value: value and value=="entry textbox content-all")
        total_share = h2.text.split()[2]
        share_fb = share_a_fb.text.strip()
        share_tw = share_a_tw.text.strip()
        share_gp = share_a_gp.text.strip()
        view_ = int(view_div.text.strip().split()[1].replace(',',''))
        text_  = div3.text.strip()
        print('URL',c+1,'Element',each,view_,convertKtoNum(total_share),share_fb,share_tw,share_gp,text_)
        process_data = create_ListofDict(each,view_,convertKtoNum(total_share),
                                         convertKtoNum(share_fb),convertKtoNum(share_tw),convertKtoNum(share_gp),text_)
        i_e_scrape.insert_one(process_data)
    except TimeoutException:
        print('Request time out at row',c)
        reqTo.append(c)
    driver.quit()
    clear_output(wait=True)